In [12]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [13]:
min_date = '20180101'

In [14]:
customers_dataset = spark.read.format("csv").option("header", "true").load("source_data/olist_customers_dataset.csv")
order_items_dataset = spark.read.format("csv").option("header", "true").load("source_data/olist_order_items_dataset.csv")
orders_dataset = spark.read.format("csv").option("header", "true").load("source_data/olist_orders_dataset.csv")
products_dataset = spark.read.format("csv").option("header", "true").load("source_data/olist_products_dataset.csv")
sellers_dataset = spark.read.format("csv").option("header", "true").load("source_data/olist_sellers_dataset.csv")

In [15]:
customers_dataset.createOrReplaceTempView('Customers')
order_items_dataset.createOrReplaceTempView('Order_Items')
orders_dataset.createOrReplaceTempView('Orders')
products_dataset.createOrReplaceTempView('Products')
sellers_dataset.createOrReplaceTempView('Sellers')


In [16]:
customers_dataset = customers_dataset.repartition(2400,customers_dataset.customer_id)

In [17]:
order_items_dataset = order_items_dataset.repartition(2400,order_items_dataset.order_id)

In [18]:
orders_dataset = orders_dataset.repartition(2400,orders_dataset.customer_id,orders_dataset.order_id)

In [19]:
order_information = orders_dataset.join(customers_dataset,(customers_dataset.customer_id==orders_dataset.customer_id),'inner')\
    .join(order_items_dataset,(order_items_dataset.order_id == orders_dataset.order_id),'inner')\
    .select(orders_dataset.order_id,\
           order_items_dataset.product_id,\
           order_items_dataset.seller_id,\
           orders_dataset.customer_id,\
           customers_dataset.customer_unique_id,\
           customers_dataset.customer_city,\
           orders_dataset.order_purchase_timestamp)

In [20]:
order_information = order_information.repartition(2400,order_information.product_id,order_information.seller_id)

In [21]:
products_dataset = products_dataset.repartition(2400,products_dataset.product_id)

In [22]:
sellers_dataset = sellers_dataset.repartition(2400,sellers_dataset.seller_id)

In [23]:
order_information = order_information.join(products_dataset,(products_dataset.product_id == order_information.product_id),'inner')\
    .join(sellers_dataset,(sellers_dataset.seller_id == order_information.seller_id),'inner')\
    .select(order_information.order_id,\
           order_information.product_id,\
           products_dataset.product_category_name,\
           order_information.seller_id,\
           sellers_dataset.seller_city,\
           order_information.customer_id,\
           order_information.customer_unique_id,\
           order_information.customer_city,\
           order_information.order_purchase_timestamp)

In [24]:
order_information.createOrReplaceTempView('order_information')

In [25]:
df_order_information = spark.sql("""
Select order_id,
product_id,
product_category_name,
seller_id,
seller_city,
customer_id,
customer_unique_id,
customer_city,
from_unixtime(unix_timestamp(order_purchase_timestamp),'yyyyMMdd') as order_purchase_ymd,
order_purchase_timestamp
from order_information
where from_unixtime(unix_timestamp(order_purchase_timestamp),'yyyyMMdd') > {}
""".format(min_date))

In [27]:
df_order_information.count()

61335

In [69]:
#df_order_information.toPandas().to_csv("destination_data/order_information.csv", header = True)